In [ ]:
# all keras import go here and base model is built for  image of size 224x224x3 using weights from imagenet
from keras.models import Sequential, Model, load_model
from keras import applications
from keras import optimizers
from keras.layers import Dropout, Flatten, Dense
try:
    import h5py
except ImportError:
    h5py = None
img_rows, img_cols, img_channel = 224, 224, 3

base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, img_channel))

In [ ]:
# add layers, dropouts,batch normalization here,using adam as optimizer here
from keras.layers.normalization import BatchNormalization
add_model_batch_dropout = Sequential()
add_model_batch_dropout.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model_batch_dropout.add(Dense(512, activation='relu'))
add_model_batch_dropout.add(BatchNormalization())
add_model_batch_dropout.add(Dropout(0.2))
add_model_batch_dropout.add(Dense(256, activation='relu'))
add_model_batch_dropout.add(BatchNormalization())
add_model_batch_dropout.add(Dropout(0.3))
add_model_batch_dropout.add(Dense(1, activation='sigmoid'))

model_batch_dropout = Model(inputs=base_model.input, outputs=add_model_batch_dropout(base_model.output))
model_batch_dropout.compile(loss='binary_crossentropy', optimizer="adam",
              metrics=['accuracy'])

model_batch_dropout.summary()

In [ ]:
#define epochs early stopping(not used in actual model,used for experiment)
from keras.callbacks import EarlyStopping
batch_size = 32
epochs = 50
      
# autosave best Model
best_model = ModelCheckpoint('batch_normalized_weights.h5', monitor='val_acc', verbose = 1, save_best_only = True)

#save model history to track accuracy and loss
history2 = model_batch_dropout.fit(x_train, y_train, batch_size=batch_size,nb_epoch=epochs,
              validation_data=(x_test, y_test),shuffle=True,callbacks = [best_model])

In [ ]:
# save predictions from the model
predictions2 = model_batch_dropout.predict(test_images)

In [ ]:
# summarize history for accuracy
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(history2.history['acc']); plt.plot(history2.history['val_acc']);
plt.title('model accuracy'); plt.ylabel('accuracy');
plt.xlabel('epoch'); plt.legend(['train', 'valid'], loc='upper left');

# summarize history for loss
plt.subplot(1, 2, 2)
plt.plot(history2.history['loss']); plt.plot(history2.history['val_loss']);
plt.title('model loss'); plt.ylabel('loss');
plt.xlabel('epoch'); plt.legend(['train', 'valid'], loc='upper left');
plt.savefig('dropout_batch.png')
plt.show()

In [ ]:
#submit to csv for kaggle score
sample_submission3 = pd.read_csv("sample_submission.csv")
for i, name in enumerate(test_names):
    sample_submission3.loc[sample_submission3['name'] == name, 'invasive'] = predictions2[i]

sample_submission3.to_csv("submit_batch_drop.csv", index=False)